In [1]:
import pandas as pd

df = pd.read_csv('vendite.csv')

In [2]:
df.head()

,transazione,data_vendita,libro,pezzi,prezzo,cliente
0,1,2017-02-04,1,1,14.9,6906
1,1,2017-02-04,5,1,12.9,6906
2,2,2017-03-03,1,1,14.9,6720
3,2,2017-03-03,3,1,14.9,6720
4,2,2017-03-03,4,1,12.9,6720


In [3]:
all(df.groupby(['transazione', 'data_vendita']).data_vendita.nunique() == 1)

True

In [4]:
df = df[['transazione', 'libro', 'cliente']]
df.head()

,transazione,libro,cliente
0,1,1,6906
1,1,5,6906
2,2,1,6720
3,2,3,6720
4,2,4,6720


In [5]:
dataset = []
for transazione in df.transazione.unique():
    libri_venduti = set()
    for idx, riga in df[df['transazione'] == transazione].iterrows():
        libri_venduti.add(riga.libro)
    dataset.append(libri_venduti)

In [6]:
dataset[:10]

[{1, 5},
 {1, 3, 4},
 {3, 5},
 {1, 2, 4},
 {2, 3, 5},
 {3, 4},
 {1, 2, 3, 4},
 {1, 2, 3},
 {1, 2, 3, 4, 5},
 {4}]

In [25]:
from collections import defaultdict


def apriori(dataset, supporto_minimo, dimensione_massima):
    """
    utilizzo
    carrelli_frequenti, supporto_carrelli = apriori(
                                                dataset,
                                                supporto_minimo,
                                                dimensione_massima)

    Parametri
    ----------
    dataset : lista di transazioni
    supporto_minimo : int
        Supporto minimo per considerare un oggetto come frequente
    dimensione_massima : int
        Dimensione massima di oggetti presenti in un carrello frequente.

    Returns
    -------
    carrelli_frequenti : lista di carrelli acquistati frequentemente
    supporto_carrelli : dizionario
        In chiave un carrello frequente di oggetti, e valore il supporto
        del carrello
    """

    carrelli = defaultdict(list)
    puntatori_transazioni = defaultdict(list)
    carrelli_frequenti = []

    for indice_transazione, transazione in enumerate(dataset):
        for libro in transazione:
            puntatori_transazioni[libro].append(indice_transazione)
            carrelli[frozenset([libro])].append(indice_transazione)

    # trasformo in frozenset per velocizzare l'elaborazione
    # e filtro solo i libri che superano il supporto minimo
    puntatori_transazioni = {
        libro: frozenset(indici_transazioni)
        for libro, indici_transazioni in puntatori_transazioni.items()
        if len(indici_transazioni) >= supporto_minimo
    }

    carrelli = {k: frozenset(v) for k, v in carrelli.items()}

    # primo livello: mantengo solo i carrelli validi
    carrelli_partenza = {
        k for k, v in carrelli.items()
        if len(v) >= supporto_minimo
    }

    for _ in range(dimensione_massima - 1):
        # questa lista_vuota che vado a definire
        # diventerà carrelli_partenza del prossimo ciclo
        nuovi_carrelli = []

        for carrello in carrelli_partenza:
            transazioni_carrello = carrelli[carrello]
            for libro, indici_transazioni in puntatori_transazioni.items():

                if libro not in carrello:
                    # ho un libro che non ho nei carrelli_frequenti!
                    # Le transazioni del nuovo carrello saranno
                    # l'intersezione tra quelle del carrello considerato
                    # più quelle in cui è presente il nuovo libro.
                    vendite_nuovo_carrello = (
                        transazioni_carrello & indici_transazioni
                    )
                    if len(vendite_nuovo_carrello) >= supporto_minimo:
                        # Se supera il paletto del supporto minimo
                        # lo aggiungo ai carrelli frequenti, altrimenti
                        # scarto lui e tutti i gruppi successivi
                        # che includono questo carrello.
                        # Il nuovo carrello sarà l'unione degli
                        # elementi distinti del carrello di
                        # partenza più il nuovo libro.
                        nuovo_carrello = frozenset(
                            carrello | frozenset([libro])
                        )
                        if nuovo_carrello not in carrelli:
                            nuovi_carrelli.append(nuovo_carrello)
                            carrelli[nuovo_carrello] = vendite_nuovo_carrello

        # ho finito un altro livello!
        if not len(nuovi_carrelli):
            # se non ho trovato nuovi carrelli frequenti
            # posso uscire
            break
        # altrimenti, aggiungo i nuovi
        # carrelli frequenti alla lista di carrelli da ritornare,
        # e imposto i carrelli trovati in questo livello
        # come base di partenza per il ciclo successivo.
        carrelli_frequenti.extend(carrelli_partenza)
        carrelli_partenza = nuovi_carrelli

    supporto_carrelli = {}
    for carrello in carrelli:
        supporto_carrelli[carrello] = float(len(carrelli[carrello]))

    return carrelli_frequenti, supporto_carrelli

In [26]:
freqset, support = apriori(dataset, 0.5, 5)

In [27]:
freqset

[frozenset({5}),
 frozenset({3}),
 frozenset({4}),
 frozenset({1}),
 frozenset({2}),
 frozenset({1, 5}),
 frozenset({3, 5}),
 frozenset({4, 5}),
 frozenset({2, 5}),
 frozenset({1, 3}),
 frozenset({3, 4}),
 frozenset({2, 3}),
 frozenset({1, 4}),
 frozenset({2, 4}),
 frozenset({1, 2}),
 frozenset({1, 3, 5}),
 frozenset({1, 4, 5}),
 frozenset({1, 2, 5}),
 frozenset({3, 4, 5}),
 frozenset({2, 3, 5}),
 frozenset({2, 4, 5}),
 frozenset({1, 3, 4}),
 frozenset({1, 2, 3}),
 frozenset({2, 3, 4}),
 frozenset({1, 2, 4}),
 frozenset({1, 3, 4, 5}),
 frozenset({1, 2, 3, 5}),
 frozenset({1, 2, 4, 5}),
 frozenset({2, 3, 4, 5}),
 frozenset({1, 2, 3, 4})]

{frozenset({1}): 249.0,
 frozenset({5}): 216.0,
 frozenset({3}): 239.0,
 frozenset({4}): 238.0,
 frozenset({2}): 232.0,
 frozenset({1, 5}): 112.0,
 frozenset({3, 5}): 111.0,
 frozenset({4, 5}): 113.0,
 frozenset({2, 5}): 98.0,
 frozenset({1, 3}): 122.0,
 frozenset({3, 4}): 116.0,
 frozenset({2, 3}): 110.0,
 frozenset({1, 4}): 127.0,
 frozenset({2, 4}): 111.0,
 frozenset({1, 2}): 122.0,
 frozenset({1, 3, 5}): 54.0,
 frozenset({1, 4, 5}): 60.0,
 frozenset({1, 2, 5}): 48.0,
 frozenset({3, 4, 5}): 55.0,
 frozenset({2, 3, 5}): 48.0,
 frozenset({2, 4, 5}): 43.0,
 frozenset({1, 3, 4}): 61.0,
 frozenset({1, 2, 3}): 54.0,
 frozenset({2, 3, 4}): 46.0,
 frozenset({1, 2, 4}): 60.0,
 frozenset({1, 3, 4, 5}): 30.0,
 frozenset({1, 2, 3, 5}): 23.0,
 frozenset({1, 2, 4, 5}): 24.0,
 frozenset({2, 3, 4, 5}): 20.0,
 frozenset({1, 2, 3, 4}): 27.0,
 frozenset({1, 2, 3, 4, 5}): 13.0}

In [36]:
from collections import namedtuple

RegolaAssociativa = namedtuple(
    'RegolaAssociativa',
    ['antecedente', 'conseguente', 'lift']
)

def genera_regola(dataset, carrelli_frequenti, supporto, lift_minimo):
    vendite = len(dataset)
    # scarto quelli con solo 1 libro
    frequenti = [f for f in carrelli_frequenti if len(f) > 1]
    for carrello in frequenti:
        for libro in carrello:
            consequente = frozenset([libro])
            antecendente = carrello - consequente

            p_ab = supporto[carrello] / supporto[antecendente]
            
            # frazione di tutte le transazioni
            # che includono il conseguente
            base = supporto[consequente] / vendite
            lift = p_ab / base
            
            if lift > lift_minimo:
                yield RegolaAssociativa(
                    antecendente,
                    consequente,
                    lift)

In [39]:
list(genera_regola(dataset, freqset, support, 1))

[RegolaAssociativa(antecedente=frozenset({1, 5}), conseguente=frozenset({4}), lift=1.012905162064826),
 RegolaAssociativa(antecedente=frozenset({1, 3, 5}), conseguente=frozenset({4}), lift=1.050420168067227),
 RegolaAssociativa(antecedente=frozenset({1, 3, 4}), conseguente=frozenset({5}), lift=1.0245901639344261),
 RegolaAssociativa(antecedente=frozenset({2, 4, 5}), conseguente=frozenset({1}), lift=1.0086859064163631),
 RegolaAssociativa(antecedente=frozenset({2, 3, 4}), conseguente=frozenset({1}), lift=1.0607647983237298)]